In [ ]:
!pip install langchain langchain-google-genai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


##1. Set Up

In [ ]:
import os
import asyncio
from typing import Any
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool, AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# Set your Gemini API key
os.environ["GEMINI_API_KEY"] = ""

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    google_api_key=os.environ["GEMINI_API_KEY"],
    model="gemini-2.0-flash",
    max_output_tokens=1024,
    temperature=0.2,
)


##2. Create Tools

In [ ]:
# Memory for storing conversions
conversion_memory = {"history": []}
tool_usage_log = []

def log_tool_usage(tool_name: str, input_data: Any):
    tool_usage_log.append({"tool": tool_name, "input": input_data})

# Temperature conversion
@tool
def convert_temperature(data: str) -> str:
    """Converts temperatures between Celsius and Fahrenheit. Input format: '100 C to F' or '212 F to C'."""
    log_tool_usage("convert_temperature", data)
    try:
        parts = data.split()
        value, unit_from, _, unit_to = parts
        value = float(value)

        if unit_from.upper() == "C" and unit_to.upper() == "F":
            result = (value * 9/5) + 32
        elif unit_from.upper() == "F" and unit_to.upper() == "C":
            result = (value - 32) * 5/9
        else:
            return "Invalid temperature conversion format."

        conversion_memory["history"].append(f"{value}{unit_from} = {result}{unit_to}")
        return f"{value}{unit_from} = {result}{unit_to}"
    except Exception as e:
        return f"Error: {str(e)}"

# Distance conversion
@tool
def convert_distance(data: str) -> str:
    """Converts distance between kilometers and miles. Input format: '10 km to mi' or '6 mi to km'."""
    log_tool_usage("convert_distance", data)
    try:
        parts = data.split()
        value, unit_from, _, unit_to = parts
        value = float(value)

        if unit_from.lower() == "km" and unit_to.lower() == "mi":
            result = value * 0.621371
        elif unit_from.lower() == "mi" and unit_to.lower() == "km":
            result = value / 0.621371
        else:
            return "Invalid distance conversion format."

        conversion_memory["history"].append(f"{value}{unit_from} = {result}{unit_to}")
        return f"{value}{unit_from} = {result}{unit_to}"
    except Exception as e:
        return f"Error: {str(e)}"

# Weight conversion
@tool
def convert_weight(data: str) -> str:
    """Converts weight between kilograms and pounds. Input format: '70 kg to lb' or '154 lb to kg'."""
    log_tool_usage("convert_weight", data)
    try:
        parts = data.split()
        value, unit_from, _, unit_to = parts
        value = float(value)

        if unit_from.lower() == "kg" and unit_to.lower() == "lb":
            result = value * 2.20462
        elif unit_from.lower() == "lb" and unit_to.lower() == "kg":
            result = value / 2.20462
        else:
            return "Invalid weight conversion format."

        conversion_memory["history"].append(f"{value}{unit_from} = {result}{unit_to}")
        return f"{value}{unit_from} = {result}{unit_to}"
    except Exception as e:
        return f"Error: {str(e)}"

# Show memory
@tool
def show_conversion_history(_: str = "") -> str:
    """Shows all past conversions done by the chatbot."""
    log_tool_usage("show_conversion_history", _)
    if not conversion_memory["history"]:
        return "No conversions yet."
    return "\n".join(conversion_memory["history"])


## 3. Bind Tools to LLM

In [ ]:
tools = [convert_temperature, convert_distance, convert_weight, show_conversion_history]
llm_with_tools = llm.bind_tools(tools)

## 4. Define Prompt

In [ ]:
MEMORY_KEY = "chat_history"

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a unit conversion assistant. Always use tools for conversion."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

## 5. Build Agent

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 6. Async Function for Queries

In [ ]:
async def run_query(user_input, chat_history):
    result = await agent_executor.ainvoke({"input": user_input, "chat_history": chat_history})
    chat_history.extend([
        HumanMessage(content=user_input),
        AIMessage(content=result["output"]),
    ])
    print("\nTools Used:")
    for usage in tool_usage_log:
        print(f"Tool: {usage['tool']}, Input: {usage['input']}")
    print("\nAssistant:", result["output"])
    return chat_history

# Initialize chat history
chat_history = []


## 7. Examples

In [ ]:
chat_history = await run_query("Convert 100 C to F", chat_history)
chat_history = await run_query("Convert 62 mi to km", chat_history)
chat_history = await run_query("Convert 70 kg to lb", chat_history)
chat_history = await run_query("Show my conversion history", chat_history)



> Entering new AgentExecutor chain...

Invoking: `convert_temperature` with `{'data': '100 C to F'}`


100.0C = 212.0F100.0C is equal to 212.0F.


> Finished chain.

Tools Used:
Tool: convert_temperature, Input: 100 C to F

Assistant: 100.0C is equal to 212.0F.



> Entering new AgentExecutor chain...

Invoking: `convert_distance` with `{'data': '62 mi to km'}`


62.0mi = 99.77935886933894km62.0mi is equal to 99.77935886933894km.


> Finished chain.

Tools Used:
Tool: convert_temperature, Input: 100 C to F
Tool: convert_distance, Input: 62 mi to km

Assistant: 62.0mi is equal to 99.77935886933894km.



> Entering new AgentExecutor chain...
70.0kg is equal to 154.32358352941436lb.


> Finished chain.

Tools Used:
Tool: convert_temperature, Input: 100 C to F
Tool: convert_distance, Input: 62 mi to km

Assistant: 70.0kg is equal to 154.32358352941436lb.



> Entering new AgentExecutor chain...

Invoking: `show_conversion_history` with `{}`


100.0C = 212.0F
62.0mi = 99.77935886933894kmO